Cours : [DLC] SQL pour Débutant(e)s Jour 7 - Synthèse et Techniques Avancées
 Lien : https://www.youtube.com/watch?v=9Aj9X-j1Yys&list=PLyh35eYRez8cAEBfi1LWN70Cs77bBvCOp&index=7
 
 Github le coin stat : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL
 Fichiers Github récupérés : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL/Jour7
 

In [1]:
import duckdb
from skimpy import skim

In [2]:
# Connection à la librairie duckdb
conn = duckdb.connect()

## Fichier des ventes

In [3]:
# Requête SQL : récupération de toutes les données du fichier .csv en DF
sales_df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/indexE/Ventes.csv', header=True)
    """
).df()

In [4]:
# Stats de la DF
skim(sales_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ int32       │ 5     │                                                          │
│ │ Number of columns │ 7      │ │ datetime64  │ 1     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA  ┃ NA %   ┃ mean    ┃ sd     ┃ p0     ┃ p25    ┃ p50   ┃ p75    ┃ p100   ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩  │
│ │ VenteID            │   0 │      0 │    50.5 │  29.01 │      1 │  25.75 │  50.5 │  75.25 │    100 │ ▇▇▇▇▇▇  │  │
│ │ ClientID           │   0 │      0 │   51.26 │  29.18 │      1 │  27.75 │  53.5 │  77.25 │     99 │ ▆▇▅▇▆▇  │  │
│ │ EmployeID          │   0 │      0 │   50.65 │   29.4 │      2 │     23 │  49.5 │  76.25 │    100 │ ▇▇▇▆▇▇  │  │
│ │ ProduitID          │   0 │      0 │   48.08 │  24.56 │      3 │     29 │    46 │  65.25 │     98 │ ▅▆▇▇▅▃  │  │
│ │ QuantiteVendue     │   0 │      0 │   48.36 │  30.17 │      1 │     22 │  50.5 │     71 │    100 │ ▇▇▃▇▃▇  │  │
│ │ MontantTotal       │   0 │      0 │    8278 │  11580 │  59.98 │   1542 │  2979 │   9249 │  48500 │ ▇▁▁  ▁  │  │
│ └────────────────────┴─────┴────────┴─────────┴────────┴────────┴────────┴───────┴────────┴────────┴─────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓  │
│ ┃ column_name      ┃ NA   ┃ NA %    ┃ first                      ┃ last                       ┃ frequency    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩  │
│ │ DateVente        │    0 │       0 │    2020-01-08 12:43:33     │    2023-12-27 15:39:42     │ None         │  │
│ └──────────────────┴──────┴─────────┴────────────────────────────┴────────────────────────────┴──────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [5]:
# Enregistrement de la DF dans une BD virtuelle (requête SQL : VIEW)
conn.register('sales_db', sales_df)

## Requêtes SQL

Classer les ventes de la plus ancienne à la plus récente en ajoutant le numéro de ligne

In [6]:
conn.execute(
    """
    -- Colonnes à récupérer
    SELECT s.*
    -- Numéro de ligne par trie croissant selon la date de vente
    , ROW_NUMBER() OVER(ORDER BY s.DateVente) AS "Numéro de ligne"
    -- BD à récupérer
    FROM sales_db s
    """
).df()

,VenteID,DateVente,ClientID,EmployeID,ProduitID,QuantiteVendue,MontantTotal,Numéro de ligne
0,5,2020-01-08 12:43:33,4,63,64,8,2399.92,1
1,62,2020-01-22 12:12:56,99,46,76,13,389.87,2
2,27,2020-02-02 10:34:59,23,19,61,14,419.86,3
3,76,2020-02-11 18:54:15,6,72,55,86,30099.14,4
4,60,2020-02-29 16:46:17,83,23,24,38,2659.62,5
...,...,...,...,...,...,...,...,...
95,93,2023-10-10 17:21:14,34,13,76,71,2129.29,96
96,15,2023-10-16 01:59:19,40,25,57,53,2119.47,97
97,10,2023-11-04 21:11:33,78,48,6,95,6649.05,98
98,48,2023-11-19 09:07:43,33,88,88,40,2799.60,99


Classer les 3 ventes les plus importantes en ajoutant le numéro de ligne

In [8]:
conn.execute(
    """
    -- Colonnes à récupérer
    SELECT s.*
    -- Numéro de lige par trie décroissant selon le montant total
    , ROW_NUMBER() OVER(ORDER BY MontantTotal DESC) AS "Numéro de ligne"
    -- BD à récupérer
    FROM sales_db s
    -- Affichage limité
    LIMIT 3
    """
).df()

,VenteID,DateVente,ClientID,EmployeID,ProduitID,QuantiteVendue,MontantTotal,Numéro de ligne
0,33,2022-07-04 07:39:21,15,97,85,97,48499.03,1
1,63,2022-04-25 17:12:46,88,9,71,92,45999.08,2
2,46,2021-01-18 12:57:15,71,36,85,91,45499.09,3
